This is a minimal example that obtains the TBR (Tritium Breeding Ratio) for a simplified fusion reactor

imports required packages

In [5]:
import openmc
import openmc_dagmc_wrapper as odw
import dagmc_h5m_file_inspector as di
import tarfile
import urllib.request
# import openmc_data_donloader todo add ability to downlad materials in a nmm material

Downloads an example h5m file made using the Paramak and cad_to_h5m python packages.

In [9]:
url = "https://github.com/fusion-energy/neutronics_workflow/archive/refs/tags/v0.0.2.tar.gz"
urllib.request.urlretrieve(url, "v0.0.2.tar.gz")

tar = tarfile.open("v0.0.2.tar.gz", "r:gz")
tar.extractall(".")
tar.close()

h5m_filename = "neutronics_workflow-0.0.2/example_02_multi_volume_cell_tally/stage_2_output/dagmc.h5m"

Creates a simple openmc point source

In [13]:
di.get_materials_from_h5m(h5m_filename)

['blanket_mat',
 'blanket_rear_wall_mat',
 'center_column_shield_mat',
 'divertor_mat',
 'firstwall_mat',
 'graveyard',
 'inboard_tf_coils_mat',
 'pf_coil_case_mat',
 'pf_coil_mat',
 'tf_coil_mat']

In [11]:
source = openmc.Source()
# sets the location of the source to x=0 y=0 z=0
source.space = openmc.stats.Point((450, 0, 0))
# sets the direction to isotropic
source.angle = openmc.stats.Isotropic()
# sets the energy distribution to 100% 14MeV neutrons
source.energy = openmc.stats.Discrete([14e6], [1])

Creates a NeutronicsModel object with the h5m file, openmc source and materials from the neutronic_material_maker database

In [18]:
# makes the neutronics model from the geometry and material allocations
my_model = odw.NeutronicsModel(
    h5m_filename=h5m_filename,
    source=source,
    materials={
         'blanket_mat':'Li4SiO4',
         'blanket_rear_wall_mat':'eurofer',
         'center_column_shield_mat':'eurofer',
         'divertor_mat':'eurofer',
         'firstwall_mat':'eurofer',
         'inboard_tf_coils_mat':'eurofer',
         'pf_coil_case_mat':'eurofer',
         'pf_coil_mat':'eurofer',
         'tf_coil_mat':'eurofer'
    },
    cell_tallies=['TBR', 'heating'],
)

Exports the xml files needed by openmc to perform the simulation. The number of batch and particles would need increasing to obtain accurate results.

In [19]:
my_model.export_xml(
    simulation_batches=2,
    simulation_particles_per_batch=100
)

/opt/openmc/openmc/mixin.py:67: IDWarning:

Another Filter instance already exists with id=1.

/opt/openmc/openmc/mixin.py:67: IDWarning:

Another Filter instance already exists with id=2.

/opt/openmc/openmc/mixin.py:67: IDWarning:

Another Filter instance already exists with id=3.

/opt/openmc/openmc/mixin.py:67: IDWarning:

Another Filter instance already exists with id=4.

/opt/openmc/openmc/mixin.py:67: IDWarning:

Another Filter instance already exists with id=5.

/opt/openmc/openmc/mixin.py:67: IDWarning:

Another Filter instance already exists with id=6.

/opt/openmc/openmc/mixin.py:67: IDWarning:

Another Filter instance already exists with id=7.

/opt/openmc/openmc/mixin.py:67: IDWarning:

Another Filter instance already exists with id=8.

/opt/openmc/openmc/mixin.py:67: IDWarning:

Another Filter instance already exists with id=9.



Simulates the model using openmc

In [21]:
statepoint_filename = my_model.simulate()


                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

extracts and processes the results from the statepoint file

In [25]:
results = odw.process_results(statepoint_filename=statepoint_filename)

processing TBR
processing blanket_mat_TBR
processing blanket_rear_wall_mat_TBR
processing center_column_shield_mat_TBR
processing divertor_mat_TBR
processing firstwall_mat_TBR
processing inboard_tf_coils_mat_TBR
processing pf_coil_case_mat_TBR
processing pf_coil_mat_TBR
processing tf_coil_mat_TBR
processing blanket_mat_heating
processing blanket_rear_wall_mat_heating
processing center_column_shield_mat_heating
processing divertor_mat_heating
processing firstwall_mat_heating
processing inboard_tf_coils_mat_heating
processing pf_coil_case_mat_heating
processing pf_coil_mat_heating
processing tf_coil_mat_heating


prints the results of the simulation

In [27]:
import pprint
pprint.pprint(results)

defaultdict(<class 'dict'>,
            {'TBR': {'result': 0.12117491318862389,
                     'std. dev.': 0.03467079662507109},
             'blanket_mat_TBR': {'result': 0.12116960749504556,
                                 'std. dev.': 0.03467046412453008},
             'blanket_mat_heating': {'MeV per source particle': {'result': 0.6875017743640932,
                                                                 'std. dev.': 0.20629788077176192}},
             'blanket_rear_wall_mat_TBR': {'result': 0.0, 'std. dev.': 0.0},
             'blanket_rear_wall_mat_heating': {'MeV per source particle': {'result': 0.0,
                                                                           'std. dev.': 0.0}},
             'center_column_shield_mat_TBR': {'result': 3.7176088965332814e-07,
                                              'std. dev.': 2.6024613099955484e-07},
             'center_column_shield_mat_heating': {'MeV per source particle': {'result': 0.29760957274781974,
 